In [1]:
import argparse
import os
import random
import scipy as sp
import pickle

import shutil
import csv
import ast

import scipy.sparse as sparse
from tqdm import tqdm
from torch import Tensor
import networkx as nx
import numpy as np
from datetime import datetime
import torch
import torch.nn as nn
from torch_geometric.data import Data

import torch.nn.functional as F
from torch_geometric.loader import DataLoader

from autoencoder import VariationalAutoEncoder, VariationalAutoEncoderWithInfoNCE
from denoise_model import DenoiseNN, p_losses, sample
from utils import linear_beta_schedule, construct_nx_from_adj, preprocess_dataset, subgraph_augment, edge_drop, \
    preprocess_dataset_with_pretrained_embedder

from torch.utils.data import Subset


2025-01-09 15:59:32.832085: I tensorflow/core/util/port.cc:111] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2025-01-09 15:59:32.833336: I tensorflow/tsl/cuda/cudart_stub.cc:28] Could not find cuda drivers on your machine, GPU will not be used.
2025-01-09 15:59:32.850882: E tensorflow/compiler/xla/stream_executor/cuda/cuda_dnn.cc:9342] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2025-01-09 15:59:32.850896: E tensorflow/compiler/xla/stream_executor/cuda/cuda_fft.cc:609] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2025-01-09 15:59:32.850909: E tensorflow/compiler/xla/stream_executor/cuda/cuda_blas.cc:1518] Unable to register cuBLAS factory: Attempting to regi

In [2]:
device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')
n_max_nodes=50
spectral_emb_dim=10
preprocess_function = preprocess_dataset
# preprocess train data, validation data and test data. Only once for the first time that you run the code. Then the appropriate .pt files will be saved and loaded.
trainset = preprocess_function("train", n_max_nodes, spectral_emb_dim)
validset = preprocess_function("valid", n_max_nodes, spectral_emb_dim)
testset = preprocess_function("test", n_max_nodes, spectral_emb_dim)

/home/antoine/Bureau/3A/altegrad/Conditional-Graph-Generation/src/utils.py:55: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  data_lst = torch.load(filename)


Dataset ./data/dataset_train.pt loaded from file
Dataset ./data/dataset_valid.pt loaded from file
Dataset ./data/dataset_test.pt loaded from file


/home/antoine/Bureau/3A/altegrad/Conditional-Graph-Generation/src/utils.py:30: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  data_lst = torch.load(filename)


In [3]:
from extract_data import STATS

data = trainset[0]
print(data.keys())
print(data.num_nodes)
print(list(zip(STATS, data.stats[0])))

['stats', 'edge_index', 'A', 'x', 'filename']
44
[('node', tensor(44.)), ('edge', tensor(406.)), ('degre', tensor(18.4545)), ('triangles', tensor(1049.)), ('g_cluster_coef', tensor(0.4315)), ('max_k_core', tensor(14.)), ('communities', tensor(4.))]


In [4]:
import torch

# Assuming you have imported these functions already
from extract_data import create_features, compute_MAE
from graph_utils import edge_index_to_dense

stats_true = []
num_nodes_batched = []
adj_matrices = []

# Collect the adjacency matrices and stats
for data in trainset:
    adj_matrices.append(edge_index_to_dense(data.edge_index, data.num_nodes))
    num_nodes_batched.append(data.num_nodes)
    stats_true.append(data.stats[0])

# Find the maximum number of nodes in the batch
max_num_nodes = max(num_nodes_batched)

# Pad the adjacency matrices to max_num_nodes
padded_adj_matrices = []
for adj_matrix, num_nodes in zip(adj_matrices, num_nodes_batched):
    # Padding the matrix to the shape of (max_num_nodes, max_num_nodes)
    padded_matrix = torch.zeros((max_num_nodes, max_num_nodes), dtype=adj_matrix.dtype)
    padded_matrix[:num_nodes, :num_nodes] = adj_matrix
    padded_adj_matrices.append(padded_matrix)

# Stack the padded adjacency matrices and stats
adj_matrices_padded = torch.stack(padded_adj_matrices, dim=0)
stats_true_tensor = torch.stack(stats_true, dim=0)


from extract_data import features_diff

features_pred = features_diff(adj_matrices_padded,torch.Tensor(num_nodes_batched).int())






In [10]:
print(stats_true_tensor.mean(dim=0), stats_true_tensor.std(dim=0))

tensor([3.0606e+01, 2.2626e+02, 1.2925e+01, 1.3899e+03, 5.0697e-01, 1.1442e+01,
        3.3475e+00]) tensor([1.1808e+01, 2.3441e+02, 1.0175e+01, 2.7951e+03, 3.2121e-01, 9.9933e+00,
        1.4503e+00])
